<a href="https://colab.research.google.com/github/VlasovArs/Recommendation_systems/blob/main/3_Demonstration_to_the_ranking_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("recdemo.csv", sep=";") #небольшой dataset

In [ ]:
df

,id,A,B,C,D,E,F
0,1,3.0,4.0,2.0,4.0,1.0,NaN
1,2,3.0,4.0,2.0,4.0,NaN,2.0
2,3,NaN,2.0,5.0,5.0,NaN,5.0
3,4,NaN,NaN,NaN,NaN,4.0,NaN
4,5,3.0,NaN,2.0,4.0,4.0,NaN
5,6,NaN,5.0,5.0,5.0,NaN,5.0
6,7,1.0,NaN,NaN,2.0,NaN,3.0
7,8,NaN,NaN,NaN,NaN,4.0,4.0


In [ ]:
df_unpivot=pd.melt(df, id_vars=['id'])
df_unpivot.head()

,id,variable,value
0,1,A,3.0
1,2,A,3.0
2,3,A,NaN
3,4,A,NaN
4,5,A,3.0


In [ ]:
df_unpivot.dropna(inplace=True)
df_unpivot.columns=['userID', 'itemID', 'rating']

In [ ]:
from surprise import Dataset
from surprise import Reader

In [ ]:
reader = Reader(rating_scale=(1, 5)) # Зададим разброс оценок
data = Dataset.load_from_df(df_unpivot[['userID', 'itemID', 'rating']], reader) #создадим объект, с которым умеет работать библиотека

In [ ]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

In [ ]:
from surprise import KNNBaseline

In [ ]:
algo = KNNBaseline(k=3, verbose=False)
algo.fit(trainset)
predictions = algo.test(testset)

In [ ]:
df_predict=pd.DataFrame(columns=['userID', 'itemID', 'rating'])
df_unpivot3 = df_unpivot.copy()
for i in predictions:
    df_unpivot3 = df_unpivot3.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
    df_predict = df_predict.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
df_unpivot3.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,2.507435
2,3.000000,4.000000,2.000000,4.000000,2.601745,2.000000
3,3.256100,2.000000,5.000000,5.000000,3.527693,5.000000
4,3.083864,4.197382,2.083864,4.083864,4.000000,3.976639
5,3.000000,3.834977,2.000000,4.000000,4.000000,3.013312
6,3.418235,5.000000,5.000000,5.000000,3.590444,5.000000
7,1.000000,3.366011,1.857717,2.000000,3.095321,3.000000
8,2.579863,3.471928,3.463240,4.463240,4.000000,4.000000


In [ ]:
df_predict_pivot=df_predict.pivot(index='userID', columns='itemID', values='rating')
df_predict_pivot

itemID,A,B,C,D,E,F
userID,,,,,,
1,NaN,NaN,NaN,NaN,NaN,2.507435
2,NaN,NaN,NaN,NaN,2.601745,NaN
3,3.256100,NaN,NaN,NaN,3.527693,NaN
4,3.083864,4.197382,2.083864,4.083864,NaN,3.976639
5,NaN,3.834977,NaN,NaN,NaN,3.013312
6,3.418235,NaN,NaN,NaN,3.590444,NaN
7,NaN,3.366011,1.857717,NaN,3.095321,NaN
8,2.579863,3.471928,3.463240,4.463240,NaN,NaN


In [ ]:
df_predict_pivot.idxmax(axis=1)

userID
1    F
2    E
3    E
4    B
5    B
6    E
7    B
8    D
dtype: object

In [ ]:
algo = KNNBaseline(k=4, sim_options = {'user_based': False }, verbose=False)
algo.fit(trainset)

predictions = algo.test(testset)

In [ ]:
df_predict=pd.DataFrame(columns=['userID', 'itemID', 'rating'])
for i in predictions:
    df_predict = df_predict.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)

In [ ]:
df_predict_pivot2=df_predict.pivot(index='userID', columns='itemID', values='rating')
df_predict_pivot2

itemID,A,B,C,D,E,F
userID,,,,,,
1,NaN,NaN,NaN,NaN,NaN,2.212165
2,NaN,NaN,NaN,NaN,2.290988,NaN
3,3.858465,NaN,NaN,NaN,4.678967,NaN
4,3.818825,4.122894,3.956402,4.261502,NaN,4.155852
5,NaN,3.337475,NaN,NaN,NaN,3.311002
6,4.698465,NaN,NaN,NaN,4.870350,NaN
7,NaN,1.789695,2.155844,NaN,2.383400,NaN
8,3.740899,4.021251,3.835183,4.150897,NaN,NaN


In [ ]:
df_predict_pivot.idxmax(axis=1)

userID
1    F
2    E
3    E
4    B
5    B
6    E
7    B
8    D
dtype: object

## Ранговые метрики

## NDSG

In [ ]:
from sklearn.metrics import ndcg_score

In [ ]:

true_relevance = np.asarray([[10, 0, 0, 1, 5]])
scores = np.asarray([[.1, .2, .3, 4, 70]])
ndcg_score(true_relevance, scores)

0.6956940443813076

In [ ]:
scores = np.asarray([[.05, 1.1, 1., .5, .0]])
ndcg_score(true_relevance, scores) # doctest: +ELLIPSIS

0.493680191377376

In [ ]:
ndcg_score(true_relevance, scores, k=4)

0.3520241100634488

## Top-K

In [ ]:
data = Dataset.load_builtin('ml-100k')

In [ ]:
from surprise.model_selection import train_test_split

In [ ]:
trainset, testset = train_test_split(data, test_size=.30) #разоббем выборку на обучение и тест

In [ ]:
algo = KNNBaseline(k=20, verbose=False)
predictions = algo.fit(trainset).test(testset)

In [ ]:
df_ratings=pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
for i in predictions:
    #if int(i.uid)==833:
    df_ratings = df_ratings.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)

In [ ]:
user456=df_ratings[df_ratings.userID=='456']

In [ ]:
def Precision_at_n(df_ratings, n=3, threshold=5):
    rel_n = set(df_ratings[df_ratings.real_rating>=threshold]['itemID'])
    t=df_ratings.sort_values(by='rating', ascending=False)[0:n]
    ret_n=set(t['itemID'])
    return len(rel_n & ret_n)/n

In [ ]:
def Avg_Precision_at_n(df_ratings, n=10,threshold=5):
    avg_p=0
    rel_n = set(df_ratings[df_ratings.real_rating>=threshold]['itemID'])

    for i in range(1,n+1):
        t=df_ratings.sort_values(by='rating', ascending=False)[0:i]
        ret_n=set(t['itemID'])
        t2=df_ratings.sort_values(by='rating', ascending=False)[i-1:i]
        ret_n2=set(t2['itemID'])
        #print(ret_n2,ret_n)
        if len((ret_n2 & rel_n))>0:
            avg_p=avg_p+len(rel_n & ret_n)/n
    return avg_p/n

In [ ]:
for i in [3,4,7,12,14]:
    print(i,Precision_at_n(user456,i))

3 0.0
4 0.25
7 0.14285714285714285
12 0.08333333333333333
14 0.07142857142857142


In [ ]:
for i in [3,4,7,12,14]:
    print(i,Avg_Precision_at_n(user456,i))

3 0.0
4 0.0625
7 0.02040816326530612
12 0.006944444444444444
14 0.00510204081632653
